## O caixeiro viajante que gosta de viajar  
<p><i>O caixeiro com gasolina infinita (e sem conciência ambiental)</i></p>


## Introdução



<p style="text-align: justify">Este problema é a história de um caixeiro que viaja entre cidade para vender sua mercadoria. Nesta variedade, o caixeiro possui gasolina infinita e gosta muito de estrada, portando, o objetivo utilizar um algoritmo genético para encontrar o trajeto entre de um conjunto de cidades, onde a distância percorrida seja a maior possível.</p>


## Objetivo



<p style="text-align: justify">Encontre o caminho de *maior* distância no problema do caixeiro viajante e mostre ele de forma gráfica.</p>

<p style="text-align: justify"><b>Considerações do experimento:</b> Considere um número $n\geq 7$ de coordenadas $(x,y)$ de cidades e que o caixeiro tenha combustível infinito. Você pode gerar as coordenadas de forma aleatória ou simplesmente usar as coordenadas que desejar. O caixeiro só anda em linha reta e apenas entre duas cidades. O caixeiro começa e termina seu trajeto na mesma cidade e, fora a cidade inicial, ele não visita nenhuma outra cidade mais de uma vez.</p>

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
from funcoes import cria_cidades
from funcoes import populacao_inicial_cv
from funcoes import funcao_objetivo_cv
from funcoes import funcao_objetivo_pop_cv
from funcoes import selecao_torneio_max # nova funcao
from funcoes import cruzamento_ordenado
from funcoes import mutacao_de_troca
from itertools import permutations
import random

<p style="text-align: justify">A primeira mão, importa-se as funções e modulos necessários para resolução do problema. Note que todas as funções utilizadas são para o Problema do Caixeiro Viajante que procura o menor trajeto possível, exeto a que realiza o torneio. Isso porque os dois problemas são de fato muito semelhantes, salvo o fato de que a situação anterior é um problema de minimização e a presente situação é um problema de maximização, assim, como o torneio é onde se define quais indivíduos seguirão para a próxima etapa do algoritmo, é necessário manipulá-lo para que selecione o maior fitness, ao invés do menor.</p>

## Códigos e discussão



In [2]:
### Constantes 

# relacionadas a busca
TAMANHO_POP = 10
NUM_GERACOES = 50
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 9
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [3]:
# criar população 

populacao = populacao_inicial_cv(TAMANHO_POP, CIDADES)

# fitness da população 

melhor_fitness = float("-inf") # Nesta linha está definido o parametro inicial para o fitness
                               # Como este é um problema de maximização, começamos com um valor
                               # extremamente baixo, assim qualquer valor acima disto será considerado
    
for i in range(NUM_GERACOES):
    
    # Seleção 
    
    fitness = funcao_objetivo_pop_cv(populacao, CIDADES) # avalia cada individou da população 
    populacao= selecao_torneio_max(populacao, fitness, NUM_COMBATENTES_NO_TORNEIO) # insere em um novo grupo os individuos com melhores avalidações
    
    # Cruzamento 
    
    # divide a população em dois grupos
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0 
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = cruzamento_ordenado(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2 
            
        contador = contador + 2
        
    # Mutação
    
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = mutacao_de_troca(individuo)
            
    # melhor individuo já visto até agora
    # lembrando que este é um problema de maximização
    
    fitness = funcao_objetivo_pop_cv(populacao, CIDADES)
    maior_fitness = max(fitness) 
    
    if maior_fitness > melhor_fitness:
        posicao = fitness.index(maior_fitness)
        melhor_individuo = populacao[posicao]
        melhor_fitness = maior_fitness

<p style="text-align: justify"> Como primeiro passo, cria-se uma população para ser trabalhada, os parâmetros como tamanho da população, número de cidades por indivíduo, número de gerações e as demais contantes do problema foram escolhidas de forma arbritária (exeto o número de cidades que está sobre a condição de ser &ge; 7). Define-se então a variável <i>melhor_fitness</i>, ela servirá de régua para selecionar os melhores resultados, por isso, ela é definida incialmente como um valor infinitamente baixo, assim, como queremos maximizar o resultado, qualquer individuo com fitness acima dela será considerado na seleção dos melhores resultados </p>

<p style="text-align: justify">Dito isto, inicia-se o primeiro laço <i><b> for </b></i>, nele, itera-se o número de gerações, assim, o algoritmo realizará um cruzamento para cada geração requisitada. Em seguida, divide-se a população em sub-grupos de <i> pais</i> e <i> maes </i>, que serão cruzados no próximo laço <i><b> for </b></i>. A variável <i> contador </i> existe para organizar os novos indivíduos, frutos do cruzamento (<i>filhos</i>), de forma ordenada, na nova população. Por fim, temos a etapa de mutação, onde um gene pode aleatóriamente ser mutado, podendo ou não gerar um melhor indivídou </p>

<p style="text-align: justify">Tendo sido realizado o cruzamento, avalia-se o fitness de cada indivíduo, o quanto ele foi bom em relação ao objetivo esperado - neste caso, a maior distância percorrida. Para isso, basta selecionar o maior fitnesse de cada população e armazerna-lo em uma variável (<i>melhor_individuo</i>, <i>melhor_fitness</i>); como estamos em um laço de repetição, se nas gerações seguintes surgirem melhores indivíduos, eles ocuparão este espaço.</p>

In [4]:
# Busca testando todas as permutações

melhor_fitness_ever = float("-inf")

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_cv(caminho, CIDADES)
    if distancia > melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho       

<p style="text-align: justify">Para obter um parâmetro de comparação, reazou-se uma busca entre todas as possibilidades de caminhos para o caixeiro, para assim avaliar o desempenho do algoritmo genético.</p>

In [6]:
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo, "com distância:", melhor_fitness)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)

Melhor individuo obtido por algoritmos genéticos:
['Cidade 5', 'Cidade 1', 'Cidade 8', 'Cidade 3', 'Cidade 7', 'Cidade 6', 'Cidade 0', 'Cidade 4', 'Cidade 2'] com distância: 6.121559720241457

Melhor individuo obtido por busca exaustiva:
('Cidade 3', 'Cidade 8', 'Cidade 6', 'Cidade 0', 'Cidade 4', 'Cidade 2', 'Cidade 5', 'Cidade 1', 'Cidade 7') com distância: 6.238056376068961


## Conclusão



<p style="text-align: justify"> Neste experimento temos, portanto, um problema de maximização resolvido por algoritmo genético. Vemos que a construção do problema não difere em muitos pontos de outros problemas relativamente mais simples, como das caixas binárias e não-binárias e descobrindo a senha. Dentre eles, a única característica essencial que muda é o formato do indivíduo. Ademais, é preciso consierar outras etapas do algoritmo e avaliar quais são as melhores estratégias, como por exemplo, neste experimento, o método de seleção usado foi o de torneio, enquanto em outros, foi usado o método da roleta. Pode-se, contudo, observar que, apesar de ser rápido e estratégico, nem sempre o algoritmo vai encotrar de fato a melhor resposta, como ocorreu na busca exaustiva.  </p>

## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

